**OpenViSUS: read from a remote dataset**

To execute this jupyter notebook have to install: OpenViSUS, numpy, matplotlib, ipyvolume

pip3 install numpy matplotlib ipyvolume

You can do it with pip3 (which uses python3):
pip3 install OpenVisus

If the installation of OpenVisus via pip does not work you have to build it and install it yourself (see OpenViSUS README), or add your build directory to the PYTHONPATH, or in your program add it to the sys path before the import:
sys.path.append('/somepath_to_your/OpenVisus/build')


In [ ]:
import os,sys
import matplotlib.pyplot as plt
import numpy
import ipyvolume

def ShowData(data):
    fig = plt.figure(figsize = (70,20))
    ax = fig.add_subplot(1,1,1)
    ax.imshow(data, origin='lower')
    plt.show()

In [ ]:
sys.path.append(r"D:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *
DbModule.attach()
db=PyDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1")
print(db.getDatasetBody().toString())

In [ ]:
max_resolution=24
logic_box=db.getLogicBox(x=[0,2048],y=[0,2048],z=[1024,2048])
data=next(db.read(logic_box=logic_box,max_resolution=max_resolution))
print("shape",data.shape,"dtype",data.dtype)

# show first slice
ShowData(data[0,:,:]) 

**Test 2D plot**

Making a 3D query for volume rendering

In [ ]:
data=next(db.read(logic_box=logic_box, max_resolution=max_resolution))
print("shape",data.shape,"dtype",data.dtype)

**Test volume rendering with iPyVolume**

In [ ]:
# this has problem on my Windows/Chrome browser
ipyvolume.quickvolshow(
    data, 
    level=[0.25, 0.75], 
    opacity=0.03, 
    level_width=0.1, 
    data_min=0, 
    data_max=255)